# Getting pmids for intramural projects

This notebook uses pages manually saved from intramural.nih.gov to pull a list of all pmids for intramural research. I've adapted code written by Dylan for the purpose of assessing collaboration.

In [30]:
import pandas as pd
import re
import glob
from pathlib import Path
import requests
from bs4 import BeautifulSoup

In [218]:
def get_pj_pmids(ipid):
    url_root = "https://intramural.nih.gov/search/"
    url = f'{url_root}searchview.taf?ipid={ipid}'
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    divs = soup.findAll('div', attrs={'id':'publications'})
    if len(divs[0].findAll(name='a')) > 0:
        if len(soup.findAll('a', attrs={'href':'#'}))>0:
            pubs = get_hidden_data(url_root, divs[0])
        else:
            sa_datas = soup.findAll('div', attrs={'style':'margin: -1.6em 0px 5px 2em;'})
            pubs = [dd for dd in sa_datas]

        if pubs:
            pubmed_ids = []
            for p in pubs:
                links = p.findAll('a', text='PubMed')
                if len(links)>0:
                    pmid = re.findall(r'pubmed\/([0-9]+)', links[0]['href'])
                    pubmed_ids.append(pmid[0])
        else:
            pubmed_ids = ''
    else:
        pubmed_ids = 'none'
    
    return(pubmed_ids)


def get_hidden_data(url_root, dd):
    if len(dd.findAll(name='a')) >0:
        aref = dd.findAll(name='a')[0]
        if 'onclick' in aref.attrs:
            show_all_search = [aa for aa in aref.attrs['onclick'].split(',') if 'searchview' in aa][0].strip("'")
            show_all_url = f'{url_root}{show_all_search}'
            show_all_page = requests.get(show_all_url)

            sa_soup = BeautifulSoup(show_all_page.text, 'html.parser')
            sa_datas = sa_soup.findAll('div', attrs={'style':'margin: -1.6em 0px 5px 2em;'})
            sa_data = [dd for dd in sa_datas]
            return sa_data
    else:
        return()

In [ ]:

    headings = [dd.text for dd in soup.findAll('div', attrs={'class':'headings'})]
    data = []
    for dd in soup.findAll('div', attrs={'class':'data'}):
        if "Show all" in dd.text:
            data.append(get_hidden_data(url_root, dd))
        else:
            data.append(dd.text)
            
    # Get project number
    headings.append('pj_num')
    data.append(soup.findAll('div', attrs={'class':'contentlabel'})[0].text)

    # Get project bibliography
    
    bib_head = [dd for dd in soup.findAll('div', attrs={'class':'headings'}) if "See Project Bibliography" in dd.text]
    if len(bib_head) > 0:
        assert len(bib_head) == 1
        headings.append('Project Bibliography')
        data.append(get_hidden_data(url_root, bib_head[0], bib=True))
    return (ipid, headings, data)


In [124]:
yrs = glob.glob('data/irp/*.htm')

In [134]:
ipid_pat = re.compile(r'\?ipid=([0-9]*)')
ipids = []
for y in yrs:
    doc = Path(y).read_text()
    ipids.extend(pd.unique(re.findall(ipid_pat, doc)))

In [189]:
res = []
for i, j in enumerate(ipids):
    res.extend(get_pj_pmids(j))

KeyError: 'onclick'

In [223]:
for k in range(i, len(ipids)):
    res.extend(get_pj_pmids(ipids[k]))

In [228]:
unique_pmids = pd.Series(pd.unique(res))

In [230]:
unique_pmids.to_csv('output/intramural_pmids.csv', index=False)

/home/riddleta/ac_knowl/ac_knowl/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [200]:
dd = divs[0].findAll(name='a')

In [203]:
len(dd)

2

In [204]:
dd.findAll(name='a')

AttributeError: ResultSet object has no attribute 'findAll'. You're probably treating a list of items like a single item. Did you call find_all() when you meant to call find()?

In [206]:
aref = soup.findAll('a', attrs={'href':'#'})[0]

In [208]:
'onclick' in aref.attrs

True